# Glacier Monitoring Benchmarking Visualization

This notebook provides comprehensive benchmarking analysis for the glacier monitoring algorithm.

## Test Structure:
1. **Cell Processing Time Breakdown** - Pie diagram of processing phases
2. **Cell Size Comparison** - Analysis of different grid sizes (1, 5, 10, 15, 20, 25 km)
3. **Hardware Metrics** - RAM, CPU%, Network, Disk usage
4. **Final Performance Metrics** - Time per km², Time per cell, Total glacier processing time

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from pathlib import Path
import geopandas as gpd

# Set style for better plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [14, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# =============================================================================
# CONFIGURATION - Set your test run folder here
# =============================================================================

OUTPUT_DIR = Path('output')

# For single test analysis, set the specific test folder
SINGLE_TEST_DIR = OUTPUT_DIR / 'test_monitored'  # Change this to your test folder

# For cell size comparison, define folders for each grid size
# Format: {grid_size_km: 'folder_name'}
CELL_SIZE_TESTS = {
    # 1: 'test_1km',     # Uncomment and set folder names as you run tests
    # 3: 'test_3km',
    # 5: 'test_5km',
    10: 'test_monitored',
    # 12: 'test_12km',
    # 15: 'test_15km',
    # 18: 'test_18km',
    # 20: 'test_20km',
    # 25: 'test_25km',
}

In [ ]:
def load_test_data(test_dir):
    """Load all data from a test directory."""
    test_path = Path(test_dir)
    data = {
        'name': test_path.name,
        'path': test_path,
        'statistics': None,
        'monitoring': None,
        'grid': None
    }
    
    # Load statistics
    stats_files = list(test_path.glob('statistics_*.json'))
    if stats_files:
        with open(stats_files[0], 'r') as f:
            data['statistics'] = json.load(f)
    
    # Load monitoring data
    monitoring_files = list(test_path.glob('proxmox_monitoring_*.json'))
    if monitoring_files:
        with open(monitoring_files[0], 'r') as f:
            data['monitoring'] = json.load(f)
    
    # Load grid
    grid_files = list(test_path.glob('processed_grid_*.geojson'))
    if grid_files:
        data['grid'] = gpd.read_file(grid_files[0])
    
    return data

# Load single test data
test_data = load_test_data(SINGLE_TEST_DIR)
print(f"Loaded test: {test_data['name']}")
print(f"  Statistics: {'✅' if test_data['statistics'] else '❌'}")
print(f"  Monitoring: {'✅' if test_data['monitoring'] else '❌'}")
print(f"  Grid: {'✅' if test_data['grid'] is not None else '❌'}")

---

# 1. Cell Processing Time Breakdown (Pie Diagram)

Shows the time distribution across different processing phases:
- STAC Query
- Zarr Loading
- NDSI Computation
- Tile Saving
- Spatial Expansion

In [ ]:
if test_data['statistics'] and 'cell_timing' in test_data['statistics'].get('results', {}):
    timing = test_data['statistics']['results']['cell_timing']
    time_dist = timing.get('time_distribution_percent', {})
    total_times = timing.get('total_times', {})
    
    # Prepare data for pie chart
    labels = ['STAC Query', 'Zarr Loading', 'NDSI Computation', 'Tile Saving', 'Spatial Expansion']
    sizes = [
        time_dist.get('stac_query', 0),
        time_dist.get('zarr_load', 0),
        time_dist.get('ndsi_compute', 0),
        time_dist.get('tile_save', 0),
        time_dist.get('spatial_expansion', 0)
    ]
    
    # Filter out zero values for cleaner pie chart
    filtered_labels = []
    filtered_sizes = []
    for label, size in zip(labels, sizes):
        if size > 0.1:  # Only show segments > 0.1%
            filtered_labels.append(label)
            filtered_sizes.append(size)
    
    # Colors for each segment
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4', '#ffeaa7']
    filtered_colors = colors[:len(filtered_labels)]
    
    # Create pie chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Pie chart
    wedges, texts, autotexts = ax1.pie(
        filtered_sizes, 
        labels=filtered_labels,
        autopct='%1.1f%%',
        colors=filtered_colors,
        explode=[0.02] * len(filtered_sizes),
        shadow=True,
        startangle=90
    )
    ax1.set_title('Cell Processing Time Breakdown', fontsize=14, fontweight='bold')
    
    # Make percentage labels more visible
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    
    # Bar chart with absolute times
    bar_labels = ['STAC Query', 'Zarr Load', 'NDSI Compute', 'Tile Save', 'Expansion']
    bar_values = [
        total_times.get('stac_query_seconds', 0),
        total_times.get('zarr_load_seconds', 0),
        total_times.get('ndsi_compute_seconds', 0),
        total_times.get('tile_save_seconds', 0),
        total_times.get('spatial_expansion_seconds', 0)
    ]
    
    bars = ax2.bar(bar_labels, bar_values, color=colors, edgecolor='white', linewidth=1.5)
    ax2.set_ylabel('Time (seconds)', fontsize=12)
    ax2.set_title('Absolute Processing Times', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='x', rotation=30)
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, val in zip(bars, bar_values):
        if val > 0:
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                    f'{val:.1f}s', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print text summary
    print("\n" + "=" * 60)
    print("CELL PROCESSING TIME BREAKDOWN")
    print("=" * 60)
    print(f"\nTime distribution:")
    print(f"  STAC query:           {time_dist.get('stac_query', 0):>6.1f}%")
    print(f"  Zarr loading:         {time_dist.get('zarr_load', 0):>6.1f}%")
    print(f"  NDSI computation:     {time_dist.get('ndsi_compute', 0):>6.1f}%")
    print(f"  Tile saving:          {time_dist.get('tile_save', 0):>6.1f}%")
    print(f"  Spatial expansion:    {time_dist.get('spatial_expansion', 0):>6.1f}%")
    print("-" * 40)
    print(f"  Total cell time:      {total_times.get('total_cell_processing_seconds', 0):.1f}s")
else:
    print("❌ No cell timing data available in statistics.")

---

# 2. Cell Size Comparison

Compare processing performance across different grid cell sizes:
- 1 km, 5 km, 10 km, 15 km, 20 km, 25 km

Shows:
- Processing time
- Number of cells
- Result grid visualization

In [ ]:
# Load all cell size test data
cell_size_data = {}

for size_km, folder_name in CELL_SIZE_TESTS.items():
    folder_path = OUTPUT_DIR / folder_name
    if folder_path.exists():
        cell_size_data[size_km] = load_test_data(folder_path)
        print(f"✅ Loaded {size_km}km test from '{folder_name}'")
    else:
        print(f"❌ Folder not found: {folder_path}")

print(f"\nLoaded {len(cell_size_data)} cell size tests")

In [ ]:
if len(cell_size_data) >= 1:
    # Prepare comparison data
    comparison = []
    for size_km, data in sorted(cell_size_data.items()):
        if data['statistics']:
            stats = data['statistics']
            config = stats.get('configuration', {})
            results = stats.get('results', {})
            timing = results.get('cell_timing', {})
            
            # Calculate snow_percentage null rate from grid
            null_rate = 0.0
            if data['grid'] is not None:
                grid = data['grid']
                processed = grid[grid['is_processed'] == True]
                if len(processed) > 0 and 'snow_percentage' in processed.columns:
                    null_count = processed['snow_percentage'].isna().sum()
                    null_rate = 100 * null_count / len(processed)
            
            comparison.append({
                'Grid Size (km)': size_km,
                'Grid Size (m)': config.get('grid_size', size_km * 1000),
                'Cells Processed': results.get('total_cells_processed', 0),
                'Total Time (s)': results.get('processing_time_seconds', 0),
                'Avg Time/Cell (s)': timing.get('average_times_per_cell', {}).get('total_seconds', 0),
                'Snow/Ice km²': results.get('snow_ice_coverage_km2', 0),
                'Snow %': results.get('snow_ice_percentage', 0),
                'Null Rate (%)': null_rate,
            })
    
    if comparison:
        comp_df = pd.DataFrame(comparison)
        print("\n" + "=" * 80)
        print("CELL SIZE COMPARISON")
        print("=" * 80)
        display(comp_df)
        
        # Visualization
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        
        sizes = comp_df['Grid Size (km)'].values
        colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(sizes)))
        
        # Total Processing Time
        ax1 = axes[0, 0]
        bars1 = ax1.bar([f"{s}km" for s in sizes], comp_df['Total Time (s)'], color=colors, edgecolor='white')
        ax1.set_ylabel('Time (seconds)')
        ax1.set_title('Total Processing Time by Grid Size', fontweight='bold')
        ax1.grid(True, alpha=0.3, axis='y')
        for bar, val in zip(bars1, comp_df['Total Time (s)']):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                    f'{val:.0f}s', ha='center', fontsize=10, fontweight='bold')
        
        # Number of Cells
        ax2 = axes[0, 1]
        bars2 = ax2.bar([f"{s}km" for s in sizes], comp_df['Cells Processed'], color=colors, edgecolor='white')
        ax2.set_ylabel('Number of Cells')
        ax2.set_title('Cells Processed by Grid Size', fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        for bar, val in zip(bars2, comp_df['Cells Processed']):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
                    str(int(val)), ha='center', fontsize=10, fontweight='bold')
        
        # Time per Cell
        ax3 = axes[0, 2]
        bars3 = ax3.bar([f"{s}km" for s in sizes], comp_df['Avg Time/Cell (s)'], color=colors, edgecolor='white')
        ax3.set_ylabel('Time (seconds)')
        ax3.set_title('Average Time per Cell', fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='y')
        for bar, val in zip(bars3, comp_df['Avg Time/Cell (s)']):
            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{val:.1f}s', ha='center', fontsize=10, fontweight='bold')
        
        # Snow Coverage
        ax4 = axes[1, 0]
        bars4 = ax4.bar([f"{s}km" for s in sizes], comp_df['Snow/Ice km²'], color=colors, edgecolor='white')
        ax4.set_ylabel('Coverage (km²)')
        ax4.set_title('Snow/Ice Coverage Detected', fontweight='bold')
        ax4.grid(True, alpha=0.3, axis='y')
        for bar, val in zip(bars4, comp_df['Snow/Ice km²']):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                    f'{val:.1f}', ha='center', fontsize=10, fontweight='bold')
        
        # Snow Percentage
        ax5 = axes[1, 1]
        bars5 = ax5.bar([f"{s}km" for s in sizes], comp_df['Snow %'], color=colors, edgecolor='white')
        ax5.set_ylabel('Snow/Ice (%)')
        ax5.set_title('Snow/Ice Percentage in Valid Area', fontweight='bold')
        ax5.grid(True, alpha=0.3, axis='y')
        for bar, val in zip(bars5, comp_df['Snow %']):
            ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                    f'{val:.1f}%', ha='center', fontsize=10, fontweight='bold')
        
        # Null Rate (Data Quality)
        ax6 = axes[1, 2]
        null_colors = ['#ff6b6b' if v > 10 else '#ffeaa7' if v > 5 else '#96ceb4' for v in comp_df['Null Rate (%)']]
        bars6 = ax6.bar([f"{s}km" for s in sizes], comp_df['Null Rate (%)'], color=null_colors, edgecolor='white')
        ax6.set_ylabel('Null Rate (%)')
        ax6.set_title('Snow% Null Rate (Data Quality)', fontweight='bold')
        ax6.grid(True, alpha=0.3, axis='y')
        ax6.axhline(y=5, color='#ffeaa7', linestyle='--', alpha=0.7, label='Warning (5%)')
        ax6.axhline(y=10, color='#ff6b6b', linestyle='--', alpha=0.7, label='Critical (10%)')
        ax6.legend(loc='upper right', fontsize=8)
        for bar, val in zip(bars6, comp_df['Null Rate (%)']):
            ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
                    f'{val:.1f}%', ha='center', fontsize=10, fontweight='bold')
        
        plt.suptitle('Grid Cell Size Comparison', fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
else:
    print("\n⚠️ Not enough cell size test data for comparison.")
    print("Run tests with different grid sizes and update CELL_SIZE_TESTS configuration.")

In [ ]:
# Visualize result grids for each cell size
grids_to_show = [(size, data) for size, data in sorted(cell_size_data.items()) if data['grid'] is not None]

if grids_to_show:
    n_grids = len(grids_to_show)
    cols = min(3, n_grids)
    rows = (n_grids + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
    if n_grids == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes if isinstance(axes, np.ndarray) else [axes]
    else:
        axes = axes.flatten()
    
    for idx, (size_km, data) in enumerate(grids_to_show):
        ax = axes[idx]
        grid = data['grid']
        
        # Plot grid with snow percentage coloring
        processed = grid[grid['is_processed'] == True]
        if 'snow_percentage' in processed.columns and not processed['snow_percentage'].isna().all():
            processed.plot(column='snow_percentage', ax=ax, legend=True, 
                          cmap='Blues', edgecolor='white', linewidth=0.5,
                          legend_kwds={'label': 'Snow %', 'shrink': 0.8})
        else:
            processed.plot(ax=ax, color='#4ecdc4', edgecolor='white', linewidth=0.5)
        
        ax.set_title(f'{size_km} km Grid ({len(processed)} cells)', fontweight='bold')
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
    
    # Hide empty subplots
    for idx in range(len(grids_to_show), len(axes)):
        axes[idx].set_visible(False)
    
    plt.suptitle('Result Grids by Cell Size', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("No grid data available for visualization.")

---

# 3. Hardware Metrics

Analyze hardware resource usage:
- RAM Usage
- CPU Utilization (%)
- Network Transfer
- Disk I/O

In [ ]:
if test_data['monitoring']:
    mon = test_data['monitoring']
    samples = pd.DataFrame(mon['samples'])
    samples['elapsed_seconds'] = samples['time'] - samples['time'].iloc[0]
    
    # Calculate derived metrics
    samples['mem_gb'] = samples['mem'] / (1024**3)
    samples['maxmem_gb'] = samples['maxmem'] / (1024**3)
    samples['netin_rate'] = samples['netin'].diff() / samples['time'].diff()
    samples['netin_mbps'] = samples['netin_rate'] * 8 / 1e6
    samples['diskread_rate'] = samples['diskread'].diff() / samples['time'].diff() / 1e6
    samples['diskwrite_rate'] = samples['diskwrite'].diff() / samples['time'].diff() / 1e6
    samples = samples.fillna(0)
    
    # Create 2x2 hardware metrics dashboard
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # RAM Usage
    ax1 = axes[0, 0]
    ax1.fill_between(samples['elapsed_seconds'], samples['mem_gb'], alpha=0.4, color='#6b8aff')
    ax1.plot(samples['elapsed_seconds'], samples['mem_gb'], color='#6b8aff', linewidth=1.5)
    ax1.axhline(y=samples['mem_gb'].mean(), color='#ff6b6b', linestyle='--', 
                label=f"Avg: {samples['mem_gb'].mean():.2f} GB")
    ax1.axhline(y=samples['maxmem_gb'].iloc[0], color='#ffaa00', linestyle=':', alpha=0.7,
                label=f"Max Available: {samples['maxmem_gb'].iloc[0]:.1f} GB")
    ax1.set_ylabel('RAM Usage (GB)', fontsize=12)
    ax1.set_xlabel('Time (seconds)', fontsize=12)
    ax1.set_title('RAM Usage', fontsize=14, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, samples['maxmem_gb'].iloc[0] * 1.1)
    
    # CPU %
    ax2 = axes[0, 1]
    ax2.fill_between(samples['elapsed_seconds'], samples['cpu_percent'], alpha=0.4, color='#00ff88')
    ax2.plot(samples['elapsed_seconds'], samples['cpu_percent'], color='#00ff88', linewidth=1.5)
    ax2.axhline(y=mon['cpu_stats']['avg_percent'], color='#ff6b6b', linestyle='--',
                label=f"Avg: {mon['cpu_stats']['avg_percent']:.1f}%")
    ax2.set_ylabel('CPU Usage (%)', fontsize=12)
    ax2.set_xlabel('Time (seconds)', fontsize=12)
    ax2.set_title('CPU Utilization', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, max(100, samples['cpu_percent'].max() * 1.1))
    
    # Network
    ax3 = axes[1, 0]
    ax3.fill_between(samples['elapsed_seconds'], samples['netin_mbps'], alpha=0.4, color='#4ecdc4')
    ax3.plot(samples['elapsed_seconds'], samples['netin_mbps'], color='#4ecdc4', linewidth=1)
    ax3.axhline(y=mon['network_transfer']['avg_speed_mbps_in'], color='#ff6b6b', linestyle='--',
                label=f"Avg: {mon['network_transfer']['avg_speed_mbps_in']:.1f} Mbps")
    ax3.set_ylabel('Network Speed (Mbps)', fontsize=12)
    ax3.set_xlabel('Time (seconds)', fontsize=12)
    ax3.set_title('Network Download Speed', fontsize=14, fontweight='bold')
    ax3.legend(loc='upper right')
    ax3.grid(True, alpha=0.3)
    
    # Disk I/O
    ax4 = axes[1, 1]
    ax4.fill_between(samples['elapsed_seconds'], samples['diskread_rate'], alpha=0.4, color='#96ceb4', label='Read')
    ax4.fill_between(samples['elapsed_seconds'], samples['diskwrite_rate'], alpha=0.4, color='#ffeaa7', label='Write')
    ax4.plot(samples['elapsed_seconds'], samples['diskread_rate'], color='#96ceb4', linewidth=1)
    ax4.plot(samples['elapsed_seconds'], samples['diskwrite_rate'], color='#ffeaa7', linewidth=1)
    ax4.set_ylabel('Disk I/O (MB/s)', fontsize=12)
    ax4.set_xlabel('Time (seconds)', fontsize=12)
    ax4.set_title('Disk I/O', fontsize=14, fontweight='bold')
    ax4.legend(loc='upper right')
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle('Hardware Resource Usage', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n" + "=" * 60)
    print("HARDWARE METRICS SUMMARY")
    print("=" * 60)
    print(f"\n💾 RAM:")
    print(f"   Min:  {samples['mem_gb'].min():.2f} GB")
    print(f"   Max:  {samples['mem_gb'].max():.2f} GB")
    print(f"   Avg:  {samples['mem_gb'].mean():.2f} GB")
    print(f"   Total: {samples['maxmem_gb'].iloc[0]:.1f} GB")
    
    print(f"\n🖥️ CPU:")
    print(f"   Min:  {mon['cpu_stats']['min_percent']:.1f}%")
    print(f"   Max:  {mon['cpu_stats']['max_percent']:.1f}%")
    print(f"   Avg:  {mon['cpu_stats']['avg_percent']:.1f}%")
    
    print(f"\n🌐 Network:")
    print(f"   Downloaded: {mon['network_transfer']['mb_in']:.1f} MB")
    print(f"   Avg Speed:  {mon['network_transfer']['avg_speed_mbps_in']:.1f} Mbps")
    print(f"   Peak Speed: {samples['netin_mbps'].max():.1f} Mbps")
    
    print(f"\n💿 Disk:")
    print(f"   Read:    {mon['disk_io']['mb_read']:.1f} MB")
    print(f"   Written: {mon['disk_io']['mb_written']:.1f} MB")
else:
    print("❌ No monitoring data available. Run with --proxmox flag to collect hardware metrics.")

---

# 4. Final Performance Metrics (Bar Charts)

Key performance indicators:
- **Time per km²** - Processing efficiency
- **Time per Cell (avg)** - Cell processing speed
- **Total Glacier Area Processing Time** - End-to-end performance

In [ ]:
# Calculate final metrics from all available tests
final_metrics = []

for size_km, data in sorted(cell_size_data.items()):
    if data['statistics']:
        stats = data['statistics']
        results = stats.get('results', {})
        timing = results.get('cell_timing', {})
        
        total_time = results.get('processing_time_seconds', 0)
        cells = results.get('total_cells_processed', 0)
        coverage_km2 = results.get('snow_ice_coverage_km2', 0)
        valid_area_km2 = results.get('total_valid_area_km2', 0)
        
        # Calculate metrics
        time_per_km2 = total_time / coverage_km2 if coverage_km2 > 0 else 0
        time_per_cell = timing.get('average_times_per_cell', {}).get('total_seconds', 
                                   total_time / cells if cells > 0 else 0)
        
        # Calculate null rate from grid
        null_rate = 0.0
        null_count = 0
        total_processed = 0
        if data['grid'] is not None:
            grid = data['grid']
            processed = grid[grid['is_processed'] == True]
            total_processed = len(processed)
            if total_processed > 0 and 'snow_percentage' in processed.columns:
                null_count = int(processed['snow_percentage'].isna().sum())
                null_rate = 100 * null_count / total_processed
        
        final_metrics.append({
            'Grid Size': f'{size_km} km',
            'Grid Size (km)': size_km,
            'Time per km² (s)': time_per_km2,
            'Avg Time per Cell (s)': time_per_cell,
            'Total Processing Time (s)': total_time,
            'Glacier Area (km²)': coverage_km2,
            'Cells Processed': cells,
            'Null Count': null_count,
            'Null Rate (%)': null_rate
        })

if final_metrics:
    metrics_df = pd.DataFrame(final_metrics)
    
    # Create bar charts
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    sizes = metrics_df['Grid Size'].values
    colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(sizes)))
    
    # Time per km²
    ax1 = axes[0, 0]
    bars1 = ax1.bar(sizes, metrics_df['Time per km² (s)'], color=colors, edgecolor='white', linewidth=2)
    ax1.set_ylabel('Time (seconds)', fontsize=12)
    ax1.set_title('Processing Time per km²', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars1, metrics_df['Time per km² (s)']):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{val:.2f}s', ha='center', fontsize=11, fontweight='bold')
    
    # Avg Time per Cell
    ax2 = axes[0, 1]
    bars2 = ax2.bar(sizes, metrics_df['Avg Time per Cell (s)'], color=colors, edgecolor='white', linewidth=2)
    ax2.set_ylabel('Time (seconds)', fontsize=12)
    ax2.set_title('Average Processing Time per Cell', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars2, metrics_df['Avg Time per Cell (s)']):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{val:.1f}s', ha='center', fontsize=11, fontweight='bold')
    
    # Total Processing Time for Glacier Area
    ax3 = axes[1, 0]
    bars3 = ax3.bar(sizes, metrics_df['Total Processing Time (s)'], color=colors, edgecolor='white', linewidth=2)
    ax3.set_ylabel('Time (seconds)', fontsize=12)
    ax3.set_title('Total Glacier Area Processing Time', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars3, metrics_df['Total Processing Time (s)']):
        mins = int(val // 60)
        secs = int(val % 60)
        label = f'{mins}m {secs}s' if mins > 0 else f'{val:.0f}s'
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                label, ha='center', fontsize=11, fontweight='bold')
    
    # Null Rate (Data Quality)
    ax4 = axes[1, 1]
    null_colors = ['#ff6b6b' if v > 10 else '#ffeaa7' if v > 5 else '#96ceb4' for v in metrics_df['Null Rate (%)']]
    bars4 = ax4.bar(sizes, metrics_df['Null Rate (%)'], color=null_colors, edgecolor='white', linewidth=2)
    ax4.set_ylabel('Null Rate (%)', fontsize=12)
    ax4.set_title('Snow% Null Rate (Data Quality)', fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    ax4.axhline(y=5, color='#ffeaa7', linestyle='--', alpha=0.7, label='Warning (5%)')
    ax4.axhline(y=10, color='#ff6b6b', linestyle='--', alpha=0.7, label='Critical (10%)')
    ax4.legend(loc='upper right')
    for bar, val, cnt in zip(bars4, metrics_df['Null Rate (%)'], metrics_df['Null Count']):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
                f'{val:.1f}%\n({int(cnt)} cells)', ha='center', fontsize=10, fontweight='bold')
    
    plt.suptitle('Final Performance Metrics', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print("\n" + "=" * 80)
    print("FINAL PERFORMANCE METRICS")
    print("=" * 80)
    display(metrics_df[['Grid Size', 'Time per km² (s)', 'Avg Time per Cell (s)', 
                        'Total Processing Time (s)', 'Glacier Area (km²)', 'Cells Processed',
                        'Null Count', 'Null Rate (%)']])
else:
    print("❌ No statistics data available for final metrics.")

In [ ]:
# Combined efficiency visualization
if final_metrics and len(final_metrics) > 1:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(sizes))
    width = 0.25
    
    # Normalize values for comparison (scale to 0-100)
    max_time_km2 = max(m['Time per km² (s)'] for m in final_metrics)
    max_time_cell = max(m['Avg Time per Cell (s)'] for m in final_metrics)
    max_total = max(m['Total Processing Time (s)'] for m in final_metrics)
    
    time_km2_norm = [100 * m['Time per km² (s)'] / max_time_km2 for m in final_metrics]
    time_cell_norm = [100 * m['Avg Time per Cell (s)'] / max_time_cell for m in final_metrics]
    total_norm = [100 * m['Total Processing Time (s)'] / max_total for m in final_metrics]
    
    bars1 = ax.bar(x - width, time_km2_norm, width, label='Time/km² (normalized)', color='#ff6b6b', alpha=0.8)
    bars2 = ax.bar(x, time_cell_norm, width, label='Time/Cell (normalized)', color='#4ecdc4', alpha=0.8)
    bars3 = ax.bar(x + width, total_norm, width, label='Total Time (normalized)', color='#45b7d1', alpha=0.8)
    
    ax.set_ylabel('Normalized Value (% of max)', fontsize=12)
    ax.set_xlabel('Grid Size', fontsize=12)
    ax.set_title('Performance Comparison Across Grid Sizes (Normalized)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(sizes)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
elif final_metrics:
    print("ℹ️ Only one grid size available. Run more tests to enable comparison chart.")

---

# Summary Report

In [ ]:
print("=" * 80)
print("BENCHMARKING SUMMARY REPORT")
print("=" * 80)

if test_data['statistics']:
    stats = test_data['statistics']
    results = stats.get('results', {})
    config = stats.get('configuration', {})
    timing = results.get('cell_timing', {})
    
    print(f"\n📋 Test Configuration:")
    print(f"   Grid Size: {config.get('grid_size', 'N/A')/1000:.0f} km")
    print(f"   Date Range: {config.get('date_start')} to {config.get('date_end')}")
    print(f"   NDSI Threshold: {config.get('ndsi_threshold')}")
    print(f"   Snow Expansion Threshold: {config.get('snow_percentage_threshold')*100:.0f}%")
    print(f"   Low Memory Mode: {config.get('low_memory', False)}")
    
    print(f"\n📊 Processing Results:")
    print(f"   Cells Processed: {results.get('total_cells_processed', 'N/A')}")
    print(f"   Iterations: {results.get('iterations', 'N/A')}")
    print(f"   Total Time: {results.get('processing_time_formatted', 'N/A')}")
    
    print(f"\n🏔️ Coverage Results:")
    print(f"   Snow/Ice Coverage: {results.get('snow_ice_coverage_km2', 0):.2f} km²")
    print(f"   Valid Area Analyzed: {results.get('total_valid_area_km2', 0):.2f} km²")
    print(f"   Snow/Ice Percentage: {results.get('snow_ice_percentage', 0):.1f}%")
    
    # Calculate null rate from grid
    if test_data['grid'] is not None:
        grid = test_data['grid']
        processed = grid[grid['is_processed'] == True]
        if len(processed) > 0 and 'snow_percentage' in processed.columns:
            null_count = int(processed['snow_percentage'].isna().sum())
            null_rate = 100 * null_count / len(processed)
            print(f"\n⚠️ Data Quality:")
            print(f"   Cells with null snow%: {null_count} / {len(processed)}")
            print(f"   Null Rate: {null_rate:.2f}%")
            if null_rate > 10:
                print(f"   Status: ❌ CRITICAL - Many cells missing data")
            elif null_rate > 5:
                print(f"   Status: ⚠️ WARNING - Some cells missing data")
            else:
                print(f"   Status: ✅ GOOD - Most cells have valid data")
    
    print(f"\n⏱️ Time Breakdown:")
    time_dist = timing.get('time_distribution_percent', {})
    print(f"   STAC Query:       {time_dist.get('stac_query', 0):>5.1f}%")
    print(f"   Zarr Loading:     {time_dist.get('zarr_load', 0):>5.1f}%")
    print(f"   NDSI Computation: {time_dist.get('ndsi_compute', 0):>5.1f}%")
    print(f"   Tile Saving:      {time_dist.get('tile_save', 0):>5.1f}%")
    print(f"   Expansion:        {time_dist.get('spatial_expansion', 0):>5.1f}%")

if test_data['monitoring']:
    mon = test_data['monitoring']
    print(f"\n💻 Hardware Usage:")
    print(f"   CPU Avg: {mon['cpu_stats']['avg_percent']:.1f}% (Max: {mon['cpu_stats']['max_percent']:.1f}%)")
    samples_df = pd.DataFrame(mon['samples'])
    mem_avg = samples_df['mem'].mean() / (1024**3)
    mem_max = samples_df['mem'].max() / (1024**3)
    print(f"   RAM Avg: {mem_avg:.2f} GB (Max: {mem_max:.2f} GB)")
    print(f"   Network: {mon['network_transfer']['mb_in']:.1f} MB downloaded")
    print(f"   Disk: {mon['disk_io']['mb_written']:.1f} MB written")

print("\n" + "=" * 80)